In [ ]:
@Tranghuynh
'''1/4/2021 synthesize all tasks for J1'''
##
import os
import sys
sys.path.append('E:/CURRENT PROJECTS/J1_check in-situ para/code/')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None

from sklearn.preprocessing import MinMaxScaler, LabelBinarizer,StandardScaler,RobustScaler, KBinsDiscretizer
from sklearn.metrics import (confusion_matrix, accuracy_score, classification_report,r2_score, mean_squared_error,
                             precision_score, multilabel_confusion_matrix,recall_score,f1_score,roc_auc_score,
                             plot_roc_curve,log_loss)
#
from sklearn.preprocessing import LabelEncoder,KBinsDiscretizer
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
#
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import DBSCAN
import seaborn as sns
import pickle
#
from yellowbrick.model_selection import feature_importances,FeatureImportances
from sklearn.utils import resample
import time
import datetime
from sklearn.inspection import permutation_importance
import function_all as fa







In [ ]:
## preprocessing steps
#1. , remove missing column/row, low concentration metal, clean data by IQR
#2. resample by time: Q, fill missing by median
#3. clustering BDSCAN
#4 select cluster, bootstrap sample by well


In [ ]:
##5 descriptive summary data
newpath='E:/CURRENT PROJECTS/J1_check in-situ para/revise_1/'
df=pd.read_csv(newpath+ 'df030.csv')
df.columns
key=['T', 'D', 'EC', 'pH', 'TH', 'TDS', 'Cl', 'NH4',
       'NO3', 'SO4', 'TOC','As', 'Mn','Fe']
var=['Temperature','Depth to water','Electrical conductivity','pH','Total hardness','Total dissolved solids','Chloride salt',
         'Ammonia Nitrogen','Nitrate Nitrogen', 'Sulfate','Total organic carbon','Arsenic', 'Manganese','Iron']
a=fa.sumdata(df,key,var)
print(a)
#
a.to_csv(newpath+'sumdf030_new.csv')

In [ ]:
##6 sprearman correlation

var=['X','Y','T','D', 'EC', 'pH', 'TH', 'TDS', 'Cl', 'NH4','NO3', 'SO4', 'TOC','As','Fe','Mn']

def heatmapcor(df,var,path,method,name):
    dt=df[var]
    corr_matrix = dt.corr(method=method)
    mask = np.zeros_like(corr_matrix, dtype=np.bool)
    mask_ut = np.triu(np.ones(corr_matrix.shape)).astype(np.bool)  # extract lower triangle matrix

    # create figure
    f, ax = plt.subplots(figsize=(5,5))
    heatmap = sns.heatmap(corr_matrix,
                          mask=mask_ut,
                          square=True,
                          linewidths=.5,
                          cmap='coolwarm',
                          cbar_kws={'shrink': .8,
                                    'ticks': [-1, -.5, 0, 0.5, 1]},
                          vmin=-1,
                          vmax=1,
                          annot=True,
                          annot_kws={'size': 4}
                          )

    # add the column names as labels
    ticks = np.arange(corr_matrix.shape[0]) + 0.5
    ax.set_xticks(ticks)
    ax.set_xticklabels(corr_matrix.columns, rotation=90, fontsize=8)
    ax.set_yticks(ticks)
    ax.set_yticklabels(corr_matrix.index, rotation=360, fontsize=8)
    #
    plt.savefig(path + str(name)+'_spearman_Hmap.png', format='png', dpi=2000)

heatmapcor(df,var,newpath,'spearman','df030')


## plot scatter by location
sns.lmplot(data=df, x='X', y='Y', hue='Cluster',height=6, aspect=.8,
                   fit_reg=False, legend=True, legend_out=True, palette="Set1")
plt.savefig(newpath + 'Cluster distribution TDS-year.png', format='png', dpi=1000)
#plot variable by time an cluster

sns.lmplot(data=df1, x='year', y='TDS', hue='Cluster',height=6, aspect=.8,
                   fit_reg=False, legend=True, legend_out=True, palette="Set1")

plt.xticks(np.arange(min(df1['year']),max(df1['year'])+1,5))
plt.savefig(newpath + 'Cluster distribution TDS-year.png', format='png', dpi=1000)
##group max value by cluster
df=pd.read_csv(newpath+'df_cluster.csv')
b=df.groupby('Cluster')['T','D', 'EC', 'pH', 'TH', 'TDS', 'Cl', 'NH4','NO3', 'SO4', 'TOC','As','Fe','Mn'].max()
print(b)
#b.to_csv(newpath+'cluster_max.csv')


In [ ]:
##7. run bootstrap model selection + plot
features=['X','Y','T','D', 'EC', 'pH', 'TH', 'TDS', 'Cl', 'NH4','NO3', 'SO4', 'TOC']
targets=['Fe','Mn','As']
scale=MinMaxScaler()

# prepare models in list
models = []
models.append(('MLP', MLPRegressor(hidden_layer_sizes=(20,10),learning_rate_init=0.01,max_iter=2000,
                                 early_stopping=True)))
models.append(('RFR',RandomForestRegressor()))
models.append(('KNR', KNeighborsRegressor()))
models.append(('SVR', SVR()))
models.append(('LR',LinearRegression()))
models.append(('GBR',GradientBoostingRegressor()))
# configure bootstrap sampling then predicting
n_iter = 100
n_size = 0.75
scaler=MinMaxScaler()
result1=fa.bootstrap_nmodel_ktarget(df,features,targets,n_iter,n_size,scaler,models) #model in list, not dict
result1.to_csv(newpath+'model_boots_df030.csv')

#boxplot compare train and test by each model
out=pd.read_csv(newpath+'model_boots_df030.csv')
name=out['Target'].unique()
#outFe=out[out['Target']=='Fe']
for i in name:
    fig, g = plt.subplots(figsize=(6,4))
    out1=out[out['Target']==i]
    out2=out1[['Models','RMSE train','RMSE test']]
    show=pd.melt(out2,id_vars='Models')
    g=sns.boxplot(x='Models',y='value',hue='variable',data=show,palette={'RMSE train': 'salmon',
                         'RMSE test':'deepskyblue'})
    #g.legend(loc='center right',bbox_to_anchor=(1.2,0.5))
    plt.savefig(newpath+ str(i)+'_select_model.png',format='png',dpi=1000)




In [ ]:

# 8. run optimization: 3.1 tuning
model=RandomForestRegressor()
for t in targets:
    start_time = time.time()
    result,bestmodel =fa.tune_RFR(model,df,features,t,n_iter,n_size,cv,scaler,rd_state=42)
    pickle.dump(bestmodel, open(newpath+str(t)+'_model_df030', 'wb'))
    end_time = time.time()
    print(f'model ran for {round((end_time -  start_time)/60.0, 2)} minutes')


In [ ]:
# 9. run FPI: 3.2
# 10 select group features based on max_feature and FPI-> run boostrap feature selection + plot
# 11. run UQ + show intervals
# 12. run treeintepreter + show boxplot comparison